In [60]:
from scipy.optimize import minimize
from sklearn.datasets import load_boston
from sklearn.base import BaseEstimator
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
import numpy as np

In [264]:
class JModel(BaseEstimator):
    def __init__(self, C=1.0):
        self.C = C
    
    def predict(self, X):
        return X @ self.W + self.b
    
    def fit(self, X, y):
        self.n = X.shape[1]
        
        def loss(y, predict, w):
            return np.mean((y - predict) ** 2) + \
                   np.abs(np.sin(w * np.pi)).mean() * self.C
        
        def fun(x):
            w, b = x[:-1], x[-1]
            return loss(y, X @ w + b, w)
        
        x0 = np.random.sample(self.n + 1) * 10 - 5
        
        res = minimize(fun, x0, tol=1e-7, method='Powell',
                       options={'xtol': 1e-6, 'ftol': 1e-6, 'maxfev': 1000000})
        
        if res.status != 0:
            print('Fail')
            
        self.W = res.x[:-1]
        self.b = res.x[-1]

        return self

In [265]:
X, y = load_boston(True)

In [275]:
model = JModel(C=1)
cross_val_score(model, X, y, scoring='neg_mean_squared_error').mean()

-120.83946645747623

In [267]:
model = Ridge(1000)
cross_val_score(model, X, y, scoring='neg_mean_squared_error').mean()

-39.238385713510453

In [268]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [272]:
for C in np.power(5.0, np.arange(-3, 10)):
    model = JModel(C=C)
    
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    loss = mean_squared_error(y_test, prediction)
    
    w = model.W
    diff = np.abs(np.round(w) - w)
    ints = (diff < 1e-5).sum()
    print('C: {:11.3f} loss: {:11.3f} integers count: {}'.format(C, loss, ints))

C:       0.008 loss:      21.818 integers count: 1
C:       0.040 loss:      22.083 integers count: 1
C:       0.200 loss:      21.693 integers count: 2
C:       1.000 loss:      20.750 integers count: 2
C:       5.000 loss:     215.103 integers count: 3
C:      25.000 loss:     139.074 integers count: 3
C:     125.000 loss:    1428.674 integers count: 5
C:     625.000 loss:    3314.174 integers count: 7
C:    3125.000 loss:    9326.370 integers count: 8
C:   15625.000 loss:   70078.264 integers count: 10
C:   78125.000 loss:   91434.458 integers count: 10
C:  390625.000 loss:  110820.833 integers count: 12
C: 1953125.000 loss:   55903.524 integers count: 13
